# setup dataset

In [1]:
# import stuff
import os
import numpy as np
import time
import pandas as pd

import torch
import torch.utils.data as data
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn


## make data.Dataset for training

In [3]:
# load files
# set your VOCdevkit path!
vocpath = "../VOCdevkit/VOC2007"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath)

# make Dataset
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(voc_classes)

# Dataloaderに入れるデータセットファイル。
# ゲットで叩くと画像とGTを前処理して出力してくれる。
train_dataset = VOCDataset(train_img_list, train_anno_list, phase = "train", transform=transform, transform_anno = transform_anno)
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

000005

000007

000009

000012

000016

000017

000019

000020

000021

000023

000024

000026

000030

000032

000033

000034

000035

000036

000039

000041

000042

000044

000046

000047

000048

000050

000051

000052

000060

000061

000063

000064

000065

000066

000072

000073

000077

000078

000081

000083

000089

000091

000093

000095

000099

000101

000102

000104

000107

000109

000110

000112

000113

000117

000118

000120

000121

000122

000123

000125

000129

000130

000131

000132

000133

000134

000138

000140

000141

000142

000143

000146

000147

000150

000153

000154

000156

000158

000159

000161

000162

000163

000164

000165

000169

000170

000171

000173

000174

000177

000180

000184

000187

000189

000190

000192

000193

000194

000198

000200

000203

000207

000208

000209

000210

000211

000214

000215

000218

000219

000220

000221

000222

000224

000225

000228

000229

000232

000233

000235

000236

000241

000242

000244

000245



004896

004897

004898

004902

004903

004905

004907

004910

004911

004912

004913

004916

004926

004928

004929

004931

004935

004936

004938

004939

004943

004946

004948

004950

004951

004953

004954

004955

004956

004958

004960

004961

004962

004963

004966

004967

004968

004972

004973

004974

004976

004977

004982

004983

004984

004985

004986

004987

004990

004991

004992

004994

004995

004997

004998

004999

005001

005003

005004

005006

005007

005014

005016

005018

005020

005023

005024

005026

005027

005028

005029

005032

005033

005036

005037

005039

005042

005045

005047

005052

005054

005055

005056

005057

005058

005061

005062

005063

005064

005065

005067

005068

005071

005072

005073

005077

005078

005079

005081

005084

005085

005086

005090

005093

005094

005097

005101

005102

005104

005107

005108

005110

005111

005114

005116

005121

005122

005124

005128

005129

005130

005131

005134

005135

005136



008477

008478

008482

008483

008484

008485

008492

008494

008495

008498

008499

008502

008503

008506

008509

008512

008513

008514

008517

008518

008519

008521

008522

008523

008524

008526

008529

008530

008533

008534

008535

008536

008541

008542

008549

008550

008553

008556

008557

008558

008559

008562

008564

008568

008572

008573

008576

008581

008582

008584

008585

008586

008587

008588

008592

008595

008596

008601

008602

008604

008606

008607

008608

008610

008612

008615

008617

008618

008620

008621

008624

008628

008633

008635

008636

008638

008639

008644

008645

008647

008653

008654

008655

008663

008665

008667

008670

008676

008680

008683

008687

008688

008690

008691

008692

008695

008698

008699

008701

008702

008706

008709

008710

008713

008716

008717

008718

008720

008722

008723

008725

008727

008728

008730

008731

008732

008733

008738

008739

008741

008742

008744

008747

008748

008749



In [4]:
# 動作の確認
batch_iterator = iter(dataloaders_dict["val"])  # イタレータに変換
images, targets = next(batch_iterator)  # 1番目の要素を取り出す
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].shape)  # ミニバッチのサイズのリスト、各要素は[n, 5]、nは物体数

torch.Size([32, 3, 300, 300])
32
torch.Size([1, 5])


# define SSD model

In [5]:
from utils.ssd import SSD

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

net = SSD(phase="train", cfg=ssd_cfg)

# SSDのweightsを設定
print("using vgg weights")
vgg_weights = torch.load("./weights/vgg16_reducedfc.pth")
net.vgg.load_state_dict(vgg_weights)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# 初期値を適応
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

print("set weights!")

using vgg weights
using: cuda:0
set weights!


In [7]:
from utils.focalloss import FocalLoss
from utils.ssd_model import match

# define loss
criterion = FocalLoss(num_classes=21)

# optim
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

In [8]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("used device：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loc_data, conf_data, dbox_list = outputs
                    # 要素数を把握
                    num_batch = loc_data.size(0)  # ミニバッチのサイズ
                    num_dbox = loc_data.size(1)  # DBoxの数 = 8732
                    num_classes = conf_data.size(2)  # クラス数 = 21
                    # get target
                    conf_t_label = torch.LongTensor(num_batch, num_dbox).to(device)
                    loc_t = torch.Tensor(num_batch, num_dbox, 4).to(device)                    
                    for idx in range(num_batch):  # ミニバッチでループ
                        # 現在のミニバッチの正解アノテーションのBBoxとラベルを取得
                        truths = targets[idx][:, :-1].to(device)  # BBox
                        # ラベル [物体1のラベル, 物体2のラベル, …]
                        labels = targets[idx][:, -1].to(device)

                        # デフォルトボックスを新たな変数で用意
                        dbox = dbox_list.to(device)

                        # 関数matchを実行し、loc_tとconf_t_labelの内容を更新する
                        # （詳細）
                        # loc_t:各DBoxに一番近い正解のBBoxの位置情報が上書きされる
                        # conf_t_label：各DBoxに一番近いBBoxのラベルが上書きされる
                        # ただし、一番近いBBoxとのjaccard overlapが0.5より小さい場合は
                        # 正解BBoxのラベルconf_t_labelは背景クラスの0とする
                        variance = [0.1, 0.2]
                        # このvarianceはDBoxからBBoxに補正計算する際に使用する式の係数です
                        match(0.5, truths, dbox,
                              variance, labels, loc_t, conf_t_label, idx)
                    
                    # compute focal loss
                    loss_l, loss_c = criterion(loc_data, loc_t, conf_data, conf_t_label)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd_focalloss_300_' +
                       str(epoch+1) + '.pth')


In [ ]:
num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

used device： cuda:0
-------------
Epoch 1/200
-------------
（train）


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


イテレーション 10 || Loss: 7.1378 || 10iter: 13.0881 sec.
イテレーション 20 || Loss: 5.9883 || 10iter: 8.4573 sec.
イテレーション 30 || Loss: 5.8273 || 10iter: 8.4384 sec.
